In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

21/10/28 11:48:51 WARN Utils: Your hostname, xtian-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.39 instead (on interface enp5s0)
21/10/28 11:48:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/28 11:48:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

21/10/28 11:52:47 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [10]:
training.show(10)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 10 rows



In [11]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [12]:
lrModel = lr.fit(training)

21/10/28 12:00:30 WARN Instrumentation: [3c3a3872] regParam is zero, which might cause numerical instability and overfitting.
21/10/28 12:00:30 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/10/28 12:00:30 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/10/28 12:00:30 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/10/28 12:00:30 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [13]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [14]:
lrModel.intercept

0.14228558260358093

In [15]:
training_summary = lrModel.summary

In [18]:
training_summary.r2

0.027839179518600154

In [19]:
training_summary.rootMeanSquaredError

10.16309157133015

In [20]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

21/10/28 12:02:58 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [26]:
train, test = all_data.randomSplit([0.7, 0.3])

In [27]:
train.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                352|
|   mean|0.11095881400596803|
| stddev| 10.054294394016889|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [28]:
test.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                149|
|   mean| 0.6016362240912029|
| stddev|  10.94271918779473|
|    min|-28.046018037776633|
|    max| 27.111027963108548|
+-------+-------------------+



In [31]:
correct_model = lr.fit(train)

21/10/28 12:04:57 WARN Instrumentation: [678ec8b5] regParam is zero, which might cause numerical instability and overfitting.


In [36]:
results = correct_model.evaluate(test)

In [38]:
results.rootMeanSquaredError

10.99120939067482

In [40]:
unlabeled_data = test.select("features")

In [41]:
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [42]:
predictions = correct_model.transform(unlabeled_data)

In [43]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   1.076033161615248|
|(10,[0,1,2,3,4,5,...| -1.2332085164949065|
|(10,[0,1,2,3,4,5,...|  1.0493803547218812|
|(10,[0,1,2,3,4,5,...|   2.679119994776905|
|(10,[0,1,2,3,4,5,...| 0.17148919351171543|
|(10,[0,1,2,3,4,5,...|  0.5745992519908688|
|(10,[0,1,2,3,4,5,...|-0.28191429848568683|
|(10,[0,1,2,3,4,5,...|  1.3347743278001265|
|(10,[0,1,2,3,4,5,...| -0.7575549270367695|
|(10,[0,1,2,3,4,5,...|-0.08287174135915353|
|(10,[0,1,2,3,4,5,...| -1.2382769516693561|
|(10,[0,1,2,3,4,5,...| -2.4243889105825422|
|(10,[0,1,2,3,4,5,...|  0.5196807418179286|
|(10,[0,1,2,3,4,5,...|-0.30324101322398617|
|(10,[0,1,2,3,4,5,...|-0.13468779687960364|
|(10,[0,1,2,3,4,5,...| -0.5434514566760601|
|(10,[0,1,2,3,4,5,...|   0.889129041990797|
|(10,[0,1,2,3,4,5,...| 0.22357961672911503|
|(10,[0,1,2,3,4,5,...| 0.10937096940402737|
|(10,[0,1,2,3,4,5,...|  -2.69666